In [1]:
import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal 

In [2]:
df1 = pd.read_csv("./MFCV-Low(1) [#1-2].csv", encoding='utf-8', sep='\t')
df1 = df1.iloc[:, 0][128:].str.split(',', expand=True)
df1 = df1.rename(columns=df1.iloc[0]).drop(df1.index[0]).reset_index(drop=True).apply(pd.to_numeric,1) # Convert object to float64

df2 = pd.read_csv("./MFCV-Low(1)(1) [#1-2].csv", encoding='utf-8', sep='\t')
df2 = df2.iloc[:, 0][128:].str.split(',', expand=True)
df2 = df2.rename(columns=df2.iloc[0]).drop(df2.index[0]).reset_index(drop=True).apply(pd.to_numeric,1) 

df3 = pd.read_csv("./MFCV-High(1) [#1-2].csv", encoding='utf-8', sep='\t')
df3 = df3.iloc[:, 0][128:].str.split(',', expand=True)
df3 = df3.rename(columns=df3.iloc[0]).drop(df3.index[0]).reset_index(drop=True).apply(pd.to_numeric,1)

df4 = pd.read_csv("./MFCV-High(1)(1) [#1-2].csv", encoding='utf-8', sep='\t')
df4 = df4.iloc[:, 0][128:].str.split(',', expand=True)
df4 = df4.rename(columns=df4.iloc[0]).drop(df4.index[0]).reset_index(drop=True).apply(pd.to_numeric,1)

df1

,VBias,C,G,Cmax,Freq,Cox
0,-5.0,1.201520e-10,2.571490e-10,1.349140e-10,1000.00,1.122861
1,-4.8,1.202140e-10,5.492040e-10,1.349260e-10,1000.00,1.122120
2,-4.6,1.202450e-10,3.387950e-10,1.349320e-10,1000.00,1.122208
3,-4.4,1.203030e-10,4.621690e-10,1.349300e-10,1000.00,1.122191
4,-4.2,1.203130e-10,7.093280e-10,1.349330e-10,1000.00,1.122319
...,...,...,...,...,...,...
3035,-4.2,1.202050e-10,1.023850e-08,NaN,13600.51,NaN
3036,-4.4,1.201690e-10,9.127930e-09,NaN,13600.51,NaN
3037,-4.6,1.201540e-10,8.512020e-09,NaN,13600.51,NaN
3038,-4.8,1.201390e-10,9.495520e-09,NaN,13600.51,NaN


In [4]:
# Testing 
excelPaths = ["./MFCV-Low(1) [#1-2].csv", "./MFCV-Low(1)(1) [#1-2].csv", "./MFCV-High(1) [#1-2].csv", "./MFCV-High(1)(1) [#1-2].csv"]

dfs = []
_Freq = []
_W = []
_V = []
_Area = 1 * 1e-3
for i in range(len(excelPaths)):
            dfs.append("df" + str(i))
            dfs[i] = pd.DataFrame()
            dfs[i] = pd.read_csv(excelPaths[i], encoding='utf-8', sep='\t')
            dfs[i] = dfs[i].iloc[:, 0][128:].str.split(',', expand=True)
            dfs[i] = dfs[i].rename(columns=dfs[i].iloc[0]).drop(dfs[i].index[0]).reset_index(drop=True).apply(pd.to_numeric, 1)  # Convert object to float64

            # Definite Value
            _Freq += sorted(list(set(dfs[i]["Freq"].tolist())))
            _V = dfs[0].query("Freq == @_Freq[0]")["VBias"].tolist()  # all V #df1.iloc[2:154, 0].tolist()  # 152 (-15 ~ 15)
            _Area = 1 * 1e-3

            
G1 = pd.DataFrame(columns = _Freq) # C
G2 = pd.DataFrame(columns = _Freq) # G

for i in range(len(_Freq)):
    _W.append(2 * (math.pi) * _Freq[i])
    listC = []
    listG = []
    for j in range(len(excelPaths)):
        listC += dfs[j].query("Freq == @_Freq[@i]")["C"].tolist()
        listG += dfs[j].query("Freq == @_Freq[@i]")["G"].tolist()

    G1[_Freq[i]] = listC
    G2[_Freq[i]] = listG
    
G1           

,1000.000,1147.257,1316.199,1510.019,1732.380,1987.486,2280.157,2615.927,3001.141,3443.081,...,1452188.000,1666033.000,1911368.000,2192831.000,2515741.000,2886202.000,3311217.000,3798817.000,4358220.000,5000000.000
0,1.201520e-10,1.202720e-10,1.202520e-10,1.202380e-10,1.202300e-10,1.202410e-10,1.202320e-10,1.202260e-10,1.202070e-10,1.202020e-10,...,1.439780e-10,1.525740e-10,1.649470e-10,1.833170e-10,2.118400e-10,2.589210e-10,3.459830e-10,4.838050e-10,7.061520e-10,-1.367360e-09
1,1.202140e-10,1.202780e-10,1.202620e-10,1.202750e-10,1.202620e-10,1.202610e-10,1.202390e-10,1.202430e-10,1.202490e-10,1.202350e-10,...,1.440190e-10,1.526160e-10,1.650000e-10,1.833100e-10,2.118870e-10,2.590020e-10,3.461140e-10,4.841360e-10,7.060790e-10,-1.368150e-09
2,1.202450e-10,1.203020e-10,1.202840e-10,1.202910e-10,1.202810e-10,1.202700e-10,1.202750e-10,1.202690e-10,1.202650e-10,1.202560e-10,...,1.440510e-10,1.526490e-10,1.650420e-10,1.834120e-10,2.119390e-10,2.590810e-10,3.462330e-10,4.842440e-10,7.059500e-10,-1.369110e-09
3,1.203030e-10,1.203170e-10,1.203200e-10,1.203080e-10,1.203010e-10,1.203060e-10,1.202870e-10,1.202900e-10,1.202780e-10,1.202930e-10,...,1.440810e-10,1.526860e-10,1.650830e-10,1.834180e-10,2.119950e-10,2.591590e-10,3.463410e-10,4.843490e-10,7.060310e-10,-1.369720e-09
4,1.203130e-10,1.203500e-10,1.203550e-10,1.203350e-10,1.203300e-10,1.203150e-10,1.203020e-10,1.203040e-10,1.202950e-10,1.202950e-10,...,1.441220e-10,1.527250e-10,1.651200e-10,1.834860e-10,2.120480e-10,2.592200e-10,3.464490e-10,4.844410e-10,7.059590e-10,-1.370440e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,1.203220e-10,1.203200e-10,1.203060e-10,1.203020e-10,1.203000e-10,1.202840e-10,1.202940e-10,1.202730e-10,1.202780e-10,1.202650e-10,...,1.440740e-10,1.526790e-10,1.650570e-10,1.834700e-10,2.119930e-10,2.591250e-10,3.462850e-10,4.845100e-10,7.068810e-10,-1.370730e-09
148,1.203050e-10,1.202730e-10,1.202790e-10,1.202830e-10,1.202850e-10,1.202760e-10,1.202560e-10,1.202510e-10,1.202490e-10,1.202510e-10,...,1.440360e-10,1.526460e-10,1.650380e-10,1.834600e-10,2.119490e-10,2.590550e-10,3.462220e-10,4.844840e-10,7.068440e-10,-1.369950e-09
149,1.202730e-10,1.202910e-10,1.202930e-10,1.202590e-10,1.202480e-10,1.202720e-10,1.202500e-10,1.202480e-10,1.202470e-10,1.202310e-10,...,1.440060e-10,1.526120e-10,1.650030e-10,1.833360e-10,2.119070e-10,2.590030e-10,3.461290e-10,4.843860e-10,7.069180e-10,-1.369340e-09
150,1.202620e-10,1.202650e-10,1.202580e-10,1.202440e-10,1.202270e-10,1.202310e-10,1.202320e-10,1.202200e-10,1.202220e-10,1.202050e-10,...,1.439850e-10,1.525900e-10,1.649780e-10,1.833150e-10,2.118620e-10,2.589420e-10,3.460270e-10,4.843470e-10,7.070930e-10,-1.368790e-09


In [6]:
global results_df_sheet1, results_df_sheet2
results_df_sheet1 = pd.DataFrame()
results_df_sheet2 = pd.DataFrame()

def getPeakXY(df, k1, k2, W):
    x = np.array(df["W"])        
    y = np.array(df["Gp_W"])
    x_New = np.linspace(min(x),max(x), 10000)
    V_XY_sheet1 = []    
    V_XY_sheet2 = []    
    q = 1.6 * 1e-19
    for k in range(k1, k2):       
        p = np.polyfit(x, y, k)
        y_New = np.polyval(p, x_New)
        #print(str(W) + "(Order " + str(k) + " - Peak-Axis) X : " + str(x_New[signal.argrelextrema(y_New,np.greater)[0]]) + ", Y : " + str(y_New[signal.argrelextrema(y_New, np.greater)]))
        
        V_XY_sheet1.append(x_New[signal.argrelextrema(y_New,np.greater)[0]][0]) # append V_X-Axis
        V_XY_sheet1.append(y_New[signal.argrelextrema(y_New, np.greater)][0])   # append V_Y-Axis
        
        
        V_XY_sheet2.append(1/x_New[signal.argrelextrema(y_New,np.greater)[0]][0]) # append V_X-Axis
        V_XY_sheet2.append(y_New[signal.argrelextrema(y_New, np.greater)][0] * (1.6 * 1e-19) / 2)   # append V_Y-Axis
    results_df_sheet1.insert(len(results_df_sheet1.columns), str(W), V_XY_sheet1, True)
    results_df_sheet2.insert(len(results_df_sheet2.columns), str(W), V_XY_sheet2, True)

title = []
deg1 = 7
deg2 = 12
for k in range(deg1, deg2):
    title.append(str(k) + "X") 
    title.append(str(k) + "Y")
results_df_sheet1["V"] = title
results_df_sheet2["V"] = title
print(title)
for i in range(len(_V)):#len(_V) #90, 100 ,1
    Gp_W = []
    _each_W = []
    for j in range(len(_Freq)): #len(_Freq)        
        Cm = G1.iat[i, j] / _Area
        Gm = G2.iat[i, j] / _Area
        Cox = (G1.iloc[:,j].max()) / _Area

        Gp_W.append( float( (_W[j] * (Cox ** 2) * Gm) / ((Gm ** 2) + (_W[j] ** 2) * (((Cm - Cox) ** 2))) ))
        _each_W.append(float(_W[j]))  

        Gp_W_Results = {
            "W": _each_W,
            "Gp_W": Gp_W,    
        }    
    

    getPeakXY(pd.DataFrame(Gp_W_Results) , deg1, deg2, _V[i])
     
writer = pd.ExcelWriter('poly4-9_Peak_XY.xlsx', engine='xlsxwriter')
# results_df.T.to_csv('poly4-9_Peak_XY.csv', encoding='utf-8', header=None) 
results_df_sheet1.T.to_excel(writer, sheet_name='Sheet1', header=None)
results_df_sheet2.T.to_excel(writer, sheet_name='Sheet2', header=None)
writer.save()

results_df_sheet1.T

['7X', '7Y', '8X', '8Y', '9X', '9Y', '10X', '10Y', '11X', '11Y']


,0,1,2,3,4,5,6,7,8,9
V,7X,7Y,8X,8Y,9X,9Y,10X,10Y,11X,11Y
-5.0,1.90833e+07,1.08142e-06,1.93e+07,1.10578e-06,1.98812e+07,1.12288e-06,2.03021e+07,1.1271e-06,2.06445e+07,1.12754e-06
-4.8,1.90707e+07,1.08126e-06,1.92969e+07,1.10631e-06,1.98717e+07,1.12326e-06,2.02989e+07,1.12747e-06,2.06162e+07,1.12768e-06
-4.6,1.90487e+07,1.08186e-06,1.92812e+07,1.10669e-06,1.98592e+07,1.12331e-06,2.02989e+07,1.12753e-06,2.06131e+07,1.12773e-06
-4.4,1.90361e+07,1.08216e-06,1.92717e+07,1.10699e-06,1.9856e+07,1.12373e-06,2.02832e+07,1.12767e-06,2.06068e+07,1.12771e-06
...,...,...,...,...,...,...,...,...,...,...
-4.2,1.90267e+07,1.08225e-06,1.92655e+07,1.10689e-06,1.98497e+07,1.12343e-06,2.02927e+07,1.12756e-06,2.06194e+07,1.12775e-06
-4.4,1.9033e+07,1.08215e-06,1.92686e+07,1.10634e-06,1.98623e+07,1.1234e-06,2.02958e+07,1.12754e-06,2.06445e+07,1.12793e-06
-4.6,1.90519e+07,1.08199e-06,1.92749e+07,1.10597e-06,1.98812e+07,1.12365e-06,2.02801e+07,1.1275e-06,2.0657e+07,1.12791e-06
-4.8,1.90644e+07,1.08145e-06,1.92875e+07,1.10583e-06,1.98812e+07,1.12329e-06,2.02989e+07,1.12748e-06,2.06445e+07,1.12791e-06
